In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import copy

from collections import defaultdict 

from synthesizer import Synthesizer
from generator import Generator
from algorithm import Algorithm

# warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 80

### Эксперимент

In [2]:
# seeds = 5 * np.arange(2, 6) + 111

#### Фиксированные параметры генератора
- series_type = "default"  // обычная линейная регрессия

- dim=10 // размерность сигнала
- low=-10, high=10 // границы весов для получения отклика
- noise_var=1 // дисперсия нормального шума
- workers_num=3 // число различных генераторов
- lower_bound=100, upper_bound=400 // границы длин каждого из кусков, получаемых генератором
- alternating=True // нет двух подряд кусков одного типа
#### Фиксированные параметры алгоритма
- from_start=False // начинаем подсчет всех потерь (и составного эксперта) не с самого начала, а после того как прошли по одному разу учатки каждого из  генераторов

- length=1500 // длина изучаемого участка временного ряда

- a=-40, b=40 // параметры алгоритма, отклики отсеиваем тоже по ним

#### Гиперпараметры эксперимента
- windows = [5, 10, 20] // варьируем длину окна обучения ($\frac12x, 1x, 2x$ соответственно, где $x$  &mdash; dim  сигнала )
- weights_function // функция начальных весов, cравниваем $\dfrac1{x^\alpha}$ для разных $\alpha$ и дефолтную $\dfrac{1}{(x+1)\ln^2(x+1)}$
- alpha_function // функция коэффицента $\alpha$ в Mixing Update, cравниваем $\dfrac1{(1 + x)^\alpha}$ для разных $\alpha$

In [3]:
from hypers import weight_hypers
from hypers import alpha_hypers 

from main import experiments
from main import df

100%|████████████████████████████████████████████████████████████████████████| 2375/2375 [00:56<00:00, 41.83it/s]


In [4]:
df

,train_window,weight_function,alpha_function,mean,random_1,random_2,random_3,random_4
1,10,1 / ((x + 1) * (ln(x + 1))^2),1 / (x + 1),93840.70,79840.65,87111.0,98238.14,110173.03
0,10,1 / (x^1.01),1 / (x + 1),66154.89,56259.47,60342.21,69245.43,78772.46
4,10,1 / (x^1.01),1 / ln(x + 1),246818.03,239549.03,197168.38,256318.41,294236.29
2,10,1 / (x^1.01),1 / e^x,73376.24,66391.23,63523.12,79793.04,83797.57
3,10,1 / e^(1/4),1 / (x + 1),307145.39,283236.02,265380.04,307638.25,372327.25
6,50,1 / ((x + 1) * (ln(x + 1))^2),1 / (x + 1),98383.65,82795.0,91051.47,106276.96,113411.17
5,50,1 / (x^1.01),1 / (x + 1),66875.93,56365.72,64306.88,69798.3,77032.83
9,50,1 / (x^1.01),1 / ln(x + 1),235982.40,231825.98,187577.84,248503.8,276021.98
7,50,1 / (x^1.01),1 / e^x,168215.47,143259.74,141616.52,178508.18,209477.44
8,50,1 / e^(1/4),1 / (x + 1),319959.50,345906.12,265813.94,294975.58,373142.38


In [9]:
# df.to_csv("df-big_windows.csv", sep='\t')

In [6]:
df_w = df[df["alpha_function"] == "1 / (x + 1)"].iloc[:, :4]
df_w

,train_window,weight_function,alpha_function,mean
1,10,1 / ((x + 1) * (ln(x + 1))^2),1 / (x + 1),93840.70
0,10,1 / (x^1.01),1 / (x + 1),66154.89
3,10,1 / e^(1/4),1 / (x + 1),307145.39
6,50,1 / ((x + 1) * (ln(x + 1))^2),1 / (x + 1),98383.65
5,50,1 / (x^1.01),1 / (x + 1),66875.93
8,50,1 / e^(1/4),1 / (x + 1),319959.50
11,100,1 / ((x + 1) * (ln(x + 1))^2),1 / (x + 1),103992.15
10,100,1 / (x^1.01),1 / (x + 1),69840.73
13,100,1 / e^(1/4),1 / (x + 1),319985.92


In [7]:
df_a = df[df["weight_function"] == "1 / (x^1.01)"].iloc[:,:4]
df_a

,train_window,weight_function,alpha_function,mean
0,10,1 / (x^1.01),1 / (x + 1),66154.89
4,10,1 / (x^1.01),1 / ln(x + 1),246818.03
2,10,1 / (x^1.01),1 / e^x,73376.24
5,50,1 / (x^1.01),1 / (x + 1),66875.93
9,50,1 / (x^1.01),1 / ln(x + 1),235982.40
7,50,1 / (x^1.01),1 / e^x,168215.47
10,100,1 / (x^1.01),1 / (x + 1),69840.73
14,100,1 / (x^1.01),1 / ln(x + 1),243948.00
12,100,1 / (x^1.01),1 / e^x,228748.57


In [8]:
df[["train_window", "mean"]].groupby("train_window").mean()

,mean
train_window,
10,157467.050
50,177883.390
100,193303.074
